In [5]:
pip install --upgrade snowflake-connector-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
import snowflake.connector

In [44]:
conn = snowflake.connector.connect(
    user= 'mnonog',
    password = 'KayaNatinToL0RD!',
    account='cejmwpt-djb91267'
    )

#https://cejmwpt-djb91267.snowflakecomputing.com

In [45]:
#cs = conn.cursor()

In [47]:
#cs.execute("YOUR SQL COMMAND")

In [48]:
cs.execute("CREATE WAREHOUSE IF NOT EXISTS my_first_warehouse")

In [ ]:
#cs.execute("USE WAREHOUSE different_warehouse_mg")

In [49]:
cs.execute("CREATE DATABASE IF NOT EXISTS testdb")

In [50]:
cs.execute("CREATE SCHEMA IF NOT EXISTS testschema")

In [51]:
cs.execute(
"CREATE OR REPLACE TABLE "    
"test_table(col1 integer, col2 string)")
cs.execute(
"INSERT INTO test_table(col1, col2) "
    		"VALUES (123, 'test string1'), (456, 'test string2')")


In [54]:
#cs.execute("PUT file:///tmp/data/file* @test_table_stage") and then cs.execute("COPY INTO test_table FROM @test_table_stage").

In [55]:
cs.execute('SELECT * FROM test_table')
print(cs.fetchmany(2))


[(123, 'test string1'), (456, 'test string2')]


In [ ]:
PUT file:///data/mydata.csv @%t1/United_States/California/Los_Angeles/2016/06/01/11/